In [1]:
import torch, os, json
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def preprecessing(data):
    copy = data
    user_name = copy["name"]
    del copy["name"]
    del copy['password']

    for d in copy: # series1['0']
      for item in copy[d]:
        del item['row']
        del item['col']
    dataset = []
    y = []
    copy = {int(k): v for k, v in copy.items()}
    for d in copy:
        for item in copy[d]:
          if len(item["acc_x"]) < 20 or len(item["acc_y"]) < 20 or len(item["acc_z"]) < 20 or len(item['gyro_x']) < 20 or len(item['gyro_y']) < 20 or len(item['gyro_z']) < 20:
            continue
          acc_x20 = item["acc_x"][-20:]
          acc_y20 = item["acc_y"][-20:]
          acc_z20 = item["acc_z"][-20:]
          gyro_x20 = item['gyro_x'][-20:]
          gyro_y20 = item['gyro_y'][-20:]
          gyro_z20 = item['gyro_z'][-20:]

          li = [acc_x20, acc_y20, acc_z20, gyro_x20, gyro_y20, gyro_z20]
          ly = [item["x"], item["y"]]
          dataset.append(li)
          y.append(ly)

    np_dataset = np.array(dataset, dtype=np.float32)
    y_dataset = np.array(y, dtype=np.float32)
    X = torch.Tensor(np_dataset)
    y = torch.Tensor(y_dataset)

    return X, y

In [2]:
def getDataLoader(X, y):
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2)
  train_dataloader = DataLoader(
      TensorDataset(X_train, y_train), batch_size=8, shuffle=True, drop_last=True
  )
  valid_dataloader = DataLoader(
    TensorDataset(X_valid, y_valid), batch_size=8, shuffle=True, drop_last=True
  )

  return train_dataloader, valid_dataloader

In [3]:
X_list, y_list = [], []

for filename in os.listdir('data'):
  if filename.endswith('.json'):
    with open(os.path.join('data', filename), 'r') as file:
      cur_data = json.load(file)
      X, y = preprecessing(cur_data)
      X_list.append(X)
      y_list.append(y)

X_tensor = torch.cat(X_list, dim=0)
y_tensor = torch.cat(y_list, dim=0)
train_dataloader, valid_dataloader = getDataLoader(X_tensor, y_tensor)

In [4]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3),
            nn.MaxPool1d(kernel_size=2),
            nn.ReLU(),
            nn.BatchNorm1d(out_channels),
        )

    def forward(self, x):
        y = self.layers(x)
        return y

class CNN1DRegression(nn.Module):
    def __init__(self, output_size):
        self.output_size = output_size
        super().__init__()

        self.blocks = nn.Sequential( # x = (8, 3, 20)
            CNNBlock(6, 64), # x = (8, 64, 9)
            CNNBlock(64, 128), # x=(8, 128, 3)
        )

        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, output_size)
        )
    def forward(self, x):
        z = self.blocks(x)
        y = self.layers(z)
        return y

In [5]:
from copy import deepcopy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN1DRegression(2).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1000
print_term = 50

best_val_loss = np.inf
lowest_epoch = np.inf
patience = 100
cur_patience = 0

In [6]:
for epoch in range(1, num_epochs+1):
  for inputs, targets in train_dataloader:
    model.train()
    inputs = inputs.to(device)
    targets = targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

  with torch.no_grad():
    model.eval()
    val_loss = 0
    for inputs, targets in valid_dataloader:
      inputs = inputs.to(device)
      targets = targets.to(device)
      outputs = model(inputs)
      val_loss += criterion(outputs, targets).item()
    val_loss /= len(valid_dataloader)
  if epoch % print_term == 0:
    print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    cur_patience = 0
    best_model = deepcopy(model.state_dict())

  else:
    cur_patience += 1
    if cur_patience >= patience:
      print(f"Early stop at epoch {epoch}, loss is {best_val_loss}")
      break

torch.save(best_model, './dlbestmodel2.pth')

Epoch 50/1000, Loss: 38.23860168457031
Epoch 100/1000, Loss: 17.239612579345703
Epoch 150/1000, Loss: 75.52000427246094
Epoch 200/1000, Loss: 22.049545288085938
Epoch 250/1000, Loss: 43.321632385253906
Epoch 300/1000, Loss: 13.821167945861816
Early stop at epoch 349, loss is 25.112794001897175
